In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense

In [2]:
# Loading the dataset
# TODO

train = pd.read_csv("data/amazon_train.csv")
valid = pd.read_csv('data/amazon_valid.csv')
test = pd.read_csv('data/amazon_test.csv')
train.head()

,UserID,ProductID,Rating,Timestamp
0,A3HICVLF4PFFMN,0594481813,5.0,2014-05-05
1,A2QBZA4S1ROX9Q,0594481813,3.0,2013-05-25
2,AT09WGFUM934H,0594481813,3.0,2013-08-31
3,AGAKHE014LQFU,0594481813,3.0,2013-09-18
4,A1S6B5QFWGVL5U,0594481813,4.0,2013-06-27


### Preprocessing 

In [3]:


sampling_strategy = {1: round(train['Rating'].value_counts()[1]*2.5),
                     2: round(train['Rating'].value_counts()[2]*3),
                     3: round(train['Rating'].value_counts()[3]*2)}

ros = RandomOverSampler(sampling_strategy=sampling_strategy, random_state=42)
X_train_ros, y_train_ros = ros.fit_resample(train[['UserID', 'ProductID']], train['Rating'])

train_ros = pd.DataFrame(X_train_ros, columns=['UserID', 'ProductID'])
train_ros['Rating'] = y_train_ros

train_ros.head()

,UserID,ProductID,Rating
0,A3HICVLF4PFFMN,0594481813,5.0
1,A2QBZA4S1ROX9Q,0594481813,3.0
2,AT09WGFUM934H,0594481813,3.0
3,AGAKHE014LQFU,0594481813,3.0
4,A1S6B5QFWGVL5U,0594481813,4.0


In [4]:
train = train_ros

le_user = LabelEncoder()
train['UserEnc'] = le_user.fit_transform(train['UserID'].values)

le_product = LabelEncoder()
train['ProductEnc'] = le_product.fit_transform(train['ProductID'].values)

valid['UserEnc'] = le_user.transform(valid['UserID'].values)
valid['ProductEnc'] = le_product.transform(valid['ProductID'].values)

test['UserEnc'] = le_user.transform(test['UserID'].values)
test['ProductEnc'] = le_product.transform(test['ProductID'].values)

In [5]:
# Scaling the Rating column

scaler = MinMaxScaler()
train['Rating'] = scaler.fit_transform(train['Rating'].values.reshape(-1, 1))
valid['Rating'] = scaler.transform(valid['Rating'].values.reshape(-1, 1))

### Model 

In [6]:

num_users = train['UserEnc'].nunique()
num_products = train['ProductEnc'].nunique()
embedding_size = 200

# User embedding input and layers
user_input = Input(shape=(1,), name='user_input')
user_embedding = Embedding(input_dim=num_users, output_dim=embedding_size, name='user_embedding')(user_input)
user_vector = Flatten(name='user_vector')(user_embedding)

# Product embedding input and layers
product_input = Input(shape=(1,), name='product_input')
product_embedding = Embedding(input_dim=num_products, output_dim=embedding_size, name='product_embedding')(product_input)
product_vector = Flatten(name='product_vector')(product_embedding)

# Dot product of user and product vectors
dot_product = Dot(axes=1, name='dot_product')([user_vector, product_vector])

# Output layer with sigmoid activation
output = Dense(1, activation='sigmoid', name='output')(dot_product)

# Create the final model
model = Model(inputs=[user_input, product_input], outputs=output, name='user_product_model')



In [7]:
# Training phase
# TODO
# Compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


Model: "user_product_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ user_input (InputLayer)       │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ product_input (InputLayer)    │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ user_embedding (Embedding)    │ (None, 1, 200)            │      18,010,000 │ user_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ product_embedding (Embedding) │ (None, 1, 200)            │       9,753,800 │ product_input[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ user_vector (Flatten)         │ (None, 200)               │               0 │ user_embedding[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ product_vector (Flatten)      │ (None, 200)               │               0 │ product_embedding[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dot_product (Dot)             │ (None, 1)                 │               0 │ user_vector[0][0],         │
│                               │                           │                 │ product_vector[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ output (Dense)                │ (None, 1)                 │               2 │ dot_product[0][0]          │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 27,763,802 (105.91 MB)

 Trainable params: 27,763,802 (105.91 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
user_ids = train['UserEnc'].values
product_ids = train['ProductEnc'].values
ratings = train['Rating'].values


valid_user_ids = valid['UserEnc'].values
valid_product_ids = valid['ProductEnc'].values
valid_ratings = valid['Rating'].values

In [9]:
history = model.fit(
    [user_ids, product_ids],  
    ratings,
    validation_data=([valid_user_ids, valid_product_ids], valid_ratings),  
    epochs=10,
    batch_size=1024
)

Epoch 1/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 374s 330ms/step - accuracy: 0.4714 - loss: 0.6474 - val_accuracy: 0.6138 - val_loss: 0.5140
Epoch 2/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 352s 313ms/step - accuracy: 0.5797 - loss: 0.3746 - val_accuracy: 0.6123 - val_loss: 0.4838
Epoch 3/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 1057s 941ms/step - accuracy: 0.5869 - loss: 0.2782 - val_accuracy: 0.6120 - val_loss: 0.4849
Epoch 4/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 361s 321ms/step - accuracy: 0.5867 - loss: 0.2784 - val_accuracy: 0.6123 - val_loss: 0.4853
Epoch 5/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 338s 301ms/step - accuracy: 0.5878 - loss: 0.2752 - val_accuracy: 0.6128 - val_loss: 0.4852
Epoch 6/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 340s 302ms/step - accuracy: 0.5871 - loss: 0.2697 - val_accuracy: 0.6126 - val_loss: 0.4860
Epoch 7/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 339s 301ms/step - accuracy: 0.5871 - loss: 0.2697 - val_accuracy: 0.6127 - val_loss: 0.4862
Epoch 8/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 340s 302ms/step - a

##### . Because  evaluation score is precision the final result  on test dataset is different ,the final score on test data is 83%

In [10]:
user_values_test = test['UserEnc'].values
product_values_test = test['ProductEnc'].values

ID = test['ProductID']
User = test['UserID']

In [11]:
# Preparing submission file
Prediction = model.predict([user_values_test,product_values_test])
rescaled_Prediction = scaler.inverse_transform(Prediction)
if rescaled_Prediction.ndim > 1:
    rescaled_Prediction = rescaled_Prediction.reshape(-1)

# Check shapes of all components to debug
print(f"Shape of User: {User.shape}")
print(f"Shape of ID: {ID.shape}")
print(f"Shape of rescaled_Prediction: {rescaled_Prediction.shape}")

predictions = np.round(rescaled_Prediction)

submission = pd.DataFrame({'UserID':User,'ProductID':ID,'Rating':predictions})
submission

2805/2805 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step
Shape of User: (89732,)
Shape of ID: (89732,)
Shape of rescaled_Prediction: (89732,)


,UserID,ProductID,Rating
0,A2Y3A341VDK37H,B00HFI55N2,5.0
1,A240FRPD4MEXND,B00KIMX4EY,5.0
2,A3SBTW3WS4IQSN,B001IF252M,4.0
3,A1QCZQTJNK3MU,B00J46VVKE,4.0
4,A16JT7E5121OAB,B00JZC972Q,4.0
...,...,...,...
89727,A37Z65SZVT0TVB,B00005AR4L,5.0
89728,A3PLX6PTM2ERKL,B00004Z6PI,5.0
89729,A3TRPVAGT3NWBS,B00005NVPW,4.0
89730,AY8Q1X7G96HV5,B000059MSI,4.0


In [12]:
submission['Rating'].value_counts()

4.0    48022
5.0    40188
3.0     1439
2.0       82
1.0        1
Name: Rating, dtype: int64

### Recommendation

In [18]:
# Number of recommendations
n_recommendations = 5

# Sample user-id 
user_id = 'A2Y3A341VDK37H'

# Step 1: Find items the user hasn't rated yet
# Get all unique products
encoded_user_id = train[train['UserID'] == user_id]['UserEnc'].iloc[0]
rated_products = train[train['UserID'] == user_id]['ProductID'].unique()
unrated_products = np.setdiff1d(train['ProductID'].unique(), rated_products)

# Step 3: Map unrated products to their encoded values
unrated_product_encodings = train[train['ProductID'].isin(unrated_products)][['ProductID', 'ProductEnc']].drop_duplicates()

In [23]:

predictions = []
user_inputs = np.array([encoded_user_id] * len(unrated_product_encodings), dtype=np.int32).reshape(-1, 1)
product_inputs = np.array(unrated_product_encodings['ProductEnc'].values, dtype=np.int32).reshape(-1, 1)

predicted_scores = model.predict_on_batch([user_inputs, product_inputs])

# Collect product IDs with their predicted scores
for i, score in enumerate(predicted_scores):
    predictions.append((unrated_product_encodings.iloc[i]['ProductID'], score[0]))

sorted_predictions = sorted(predictions, key=lambda x: x[1], reverse=True)
top_n_recommendations = [product[0] for product in sorted_predictions[:n_recommendations]]

print("Top recommendations for user", user_id, ":", top_n_recommendations)

Top recommendations for user A2Y3A341VDK37H : ['B00068NUOE', 'B004286VMW', 'B000YAX13Y', 'B00906DHLE', 'B002JVC96Q']
